In [1]:
import numpy as np
from alabtools.phasing import TakeiPhaser
from alabtools.imaging import CtFile

In [11]:
ct = CtFile('ct_test.ct')  # load the ct file
print(ct.coordinates.shape)

(201, 32, 1, 9, 3)


In [12]:
controller_config = {'parallel': {'controller': 'serial'}}
phaser = TakeiPhaser(ct, controller_config, ncopy=2)

Using a <class 'alabtools.parallel.parallel_controller.SerialController'> controller.


In [13]:
phaser.phasing()

(SERIAL): 100%|██████████| 201/201 [00:02<00:00, 78.89it/s] 


In [6]:
phaser.phase

array([[[2., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 2., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [2., 1., 0., ..., 0., 0., 0.],
        [1., 2., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 2., 0., ..., 0., 0., 0.],
        [2., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 2., 1., ..., 0., 0., 0.],
        [1., 2., 0., ..., 0., 0., 0.]],

       [[2., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 2., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [2., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [2., 1., 0., ..., 0., 0., 0.],
        [1., 2., 0., ..., 0., 0., 0.],
        ...,
        [2., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
# print indices where ct.nspot is maximum
idx = np.where(ct.nspot == ct.nspot_max)  # find the maximum
print(phaser.phase[0, 0])
print(phaser.phase[idx[0], idx[1]])

[2. 0. 0. 0. 0. 0. 0. 0. 0.]
[[1. 2. 1. 1. 1. 1. 2. 1. 2.]]


I can create a phs array (ncell, ndomains) fill it with these cell/chr values.

Then I can just select the two copies as [phs == 1] and [phs == 2] and create a new ct file with the phases separated.

I probably have to find the ncopy_max and nspot_max in a for loop and fix the new ct file.

In [24]:
ncell = 6
ndomain = 21
nspot_max = 4
# create a random numpy boolean array of shape (ncell, ndomain, nspot_max)
mask = np.random.randint(2, size=(ncell, ndomain, nspot_max), dtype=bool)
print(mask.shape)

(6, 21, 4)


In [25]:
mask_expanded = np.expand_dims(mask, axis=2)
print(mask.shape, mask_expanded.shape)

(6, 21, 4) (6, 21, 1, 4)


In [26]:
# check that mask_expanded is the same as mask
print(np.array_equal(mask[:, :, :], mask_expanded[:, :, 0, :]))

True


In [27]:
# include an additional axis of length 3 at the end,
# thus from shape (6, 21, 1, 4) to (6, 21, 1, 4, 3)
mask_expanded = np.expand_dims(mask_expanded, axis=4)
mask_expanded = np.repeat(mask_expanded, repeats=3, axis=4)
print(mask_expanded.shape)

(6, 21, 1, 4, 3)


In [28]:
# check that mask_expanded is the same as mask
for i in range(3):
    print(np.array_equal(mask[:, :, :], mask_expanded[:, :, 0, :, i]))

True
True
True


In [50]:
import numpy as np

# create an example array a
a = np.random.randint(0, 5, size=(3, 4, 10))

b = np.zeros((a.shape[0], a.shape[1]), dtype=int)

for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        unq = np.unique(a[i, j, :])
        # drop the zero value
        unq = unq[unq != 0]
        b[i, j] = len(unq)
        print(a[i, j, :], np.unique(a[i, j, :]), b[i, j])


[3 3 0 3 4 4 0 3 1 1] [0 1 3 4] 3
[2 1 3 2 1 0 4 3 3 2] [0 1 2 3 4] 4
[3 0 2 0 0 2 2 0 4 2] [0 2 3 4] 3
[0 4 0 3 0 3 4 3 2 1] [0 1 2 3 4] 4
[1 0 1 1 3 4 1 2 1 0] [0 1 2 3 4] 4
[0 3 0 1 0 4 3 0 0 0] [0 1 3 4] 3
[0 4 2 1 2 1 0 1 0 2] [0 1 2 4] 3
[3 4 3 4 3 2 2 2 2 4] [2 3 4] 3
[4 2 4 4 0 3 4 1 3 2] [0 1 2 3 4] 4
[0 1 4 1 4 3 0 3 4 2] [0 1 2 3 4] 4
[3 0 4 3 0 3 1 2 2 3] [0 1 2 3 4] 4
[2 3 2 3 0 3 2 2 0 4] [0 2 3 4] 3


In [ ]:
# c = length of non-zero elements in b on axis=2
c = np.count_nonzero(b, axis=2)
for i in range(3):
    for j in range(4):
        print(np.unique(a[i, j, :]), b[i, j, :])